In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126211") # UPDATE THIS FOR NEW WORKSPACE
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126211
Azure region: southcentralus
Subscription id: ebee3a56-4c54-406a-b732-174015826780
Resource group: aml-quickstarts-126211


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "udacity-ml-comp1" # NEEDS TO BE UNIQUE!

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster")
except ComputeTargetException:    
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", 
                                                           max_nodes=4,
                                                          idle_seconds_before_scaledown=1200)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(1,5),
                             "--max_iter": choice(10,50,100) }) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".",entry_script="train.py", compute_target=cpu_cluster) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, policy=policy,
                                    primary_metric_name="Accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs="4",
                                    max_concurrent_runs=4) ### YOUR CODE HERE ###

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config) ### YOUR CODE HERE ###
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib

# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

#Get list of associated filesnames.
print(best_run.get_file_names())

# Download the best run model
best_run.download_file('outputs/model.joblib','outputs/mymodel.joblib')

# Register best model to Azure ML
model = best_run.register_model(model_name='sklearn-banking',model_path='outputs/model.joblib')
### YOUR CODE HERE ###

['--C', '1.5443815695995275', '--max_iter', '50']
['azureml-logs/55_azureml-execution-tvmps_cfc3c64ecb572fcc09fa3e45fccbcbc68977646aa3893efc83da10b4c7216001_d.txt', 'azureml-logs/65_job_prep-tvmps_cfc3c64ecb572fcc09fa3e45fccbcbc68977646aa3893efc83da10b4c7216001_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_cfc3c64ecb572fcc09fa3e45fccbcbc68977646aa3893efc83da10b4c7216001_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/103_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv") ### YOUR CODE HERE ###

### YOUR CODE HERE ###

In [4]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###)

# Split into train and test sets.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

# Concatenate the training features and labels.
import pandas as pd
training_data = pd.concat([x_train,y_train],axis=1)
training_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
12909,46,1,0,0,0,5,4,194,2,999,...,1,1,0,0,0,0,0,0,0,0
8421,39,1,0,1,0,9,5,293,1,3,...,0,0,0,1,0,0,0,0,0,1
18029,33,0,0,1,0,8,2,99,6,999,...,0,0,0,0,0,0,0,1,0,0
23392,45,1,0,0,1,7,1,93,1,999,...,0,0,0,0,0,0,0,1,0,0
28305,50,1,0,1,0,11,1,124,5,999,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# Register dataframe as a dataset for inputting into the automl.
from azureml.core import Dataset
target = ws.get_default_datastore()
TabularDatasetFactory.register_pandas_dataframe(training_data,target,name='automl')
dataset = Dataset.get_by_name(ws, name='automl')

WARNING - Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/e8ec7b1e-136c-40cb-8385-885294ca6d2a/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [6]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target = cpu_cluster,
    iterations=20)

In [7]:
# Submit your automl run
exp = Experiment(workspace=ws, name="udacity-project-automl")
run = exp.submit(automl_config, show_output=True)
RunDetails(run).show()
### YOUR CODE HERE ###

Running on remote.
Running on remote compute: udacity-ml-comp1
Parent Run ID: AutoML_dbd421f2-8851-4bdb-8bba-3602d58b7d52

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-------------------------------

In [11]:
# Retrieve and save your best automl model.
best_run, model = run.get_output()
print(best_run)
best_run.register_model(model_name='automl-best',model_path='outputs/model.pkl')
### YOUR CODE HERE ###

Run(Experiment: udacity-project-automl,
Id: AutoML_dbd421f2-8851-4bdb-8bba-3602d58b7d52_18,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-126211', subscription_id='ebee3a56-4c54-406a-b732-174015826780', resource_group='aml-quickstarts-126211'), name=automl-best, id=automl-best:1, version=1, tags={}, properties={})

In [12]:
# clean up the computer resouces
cpu_cluster.delete()